In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
!pip install ucimlrepo

  Obtaining dependency information for ucimlrepo from https://files.pythonhosted.org/packages/3e/4a/ecc3456479d687202b34ee42317c3a63e09793c9409a720052d38356431a/ucimlrepo-0.0.3-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [30]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets 

X.fillna(value=0, inplace=True)

# metadata 
print(breast_cancer_wisconsin_original.metadata) 
  
# variable information 
print(breast_cancer_wisconsin_original.variables) 

{'uci_id': 15, 'name': 'Breast Cancer Wisconsin (Original)', 'repository_url': 'https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original', 'data_url': 'https://archive.ics.uci.edu/static/public/15/data.csv', 'abstract': 'Original Wisconsin Breast Cancer Database', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 699, 'num_features': 9, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['Class'], 'index_col': ['Sample_code_number'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1990, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5HP4Z', 'creators': ['WIlliam Wolberg'], 'intro_paper': None, 'additional_info': {'summary': "Samples arrive periodically as Dr. Wolberg reports his clinical cases. The database therefore reflects this chronological grouping of the data. This grouping information appears immediately below, having been removed fro

/var/folders/_n/lz98rzln403f9jgxhtvhf_wr0000gn/T/ipykernel_79145/2362441507.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(value=0, inplace=True)


In [31]:
print(X.describe)
print(y.describe)

<bound method NDFrame.describe of      Clump_thickness  Uniformity_of_cell_size  Uniformity_of_cell_shape  \
0                  5                        1                         1   
1                  5                        4                         4   
2                  3                        1                         1   
3                  6                        8                         8   
4                  4                        1                         1   
..               ...                      ...                       ...   
694                3                        1                         1   
695                2                        1                         1   
696                5                       10                        10   
697                4                        8                         6   
698                4                        8                         8   

     Marginal_adhesion  Single_epithelial_cell_size  Bare_nuclei 

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)

In [33]:
min_train = X_train.min()
range_train = (X_train - min_train).max()
X_train_scaled = (X_train - min_train)/range_train

min_test = X_test.min()
range_test = (X_test - min_test).max()
X_test_scaled = (X_test - min_test)/range_test

In [41]:
X_train_scaled.head()

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
129,0.000000,0.000000,0.000000,0.000000,1.000000,0.1,0.000000,0.000000,0.000000
574,1.000000,0.888889,0.666667,0.222222,0.333333,0.2,0.666667,0.666667,0.000000
445,0.111111,0.000000,0.000000,0.000000,0.111111,0.1,0.000000,0.000000,0.000000
488,0.222222,0.555556,0.333333,1.000000,0.222222,0.3,0.222222,0.333333,0.000000
605,1.000000,0.444444,1.000000,0.222222,0.444444,0.8,0.666667,0.777778,0.222222


In [36]:
import numpy as np

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

svc_model = SVC()
svc_model.fit(X_train_scaled, np.ravel(y_train))
y_predict = svc_model.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_predict)

In [44]:
cm

array([[86,  5],
       [ 2, 47]])

In [39]:
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [0.1, 1, 10, 100], "gamma": [1, 0.1, 0.01, 0.001], "kernel": ["rbf"]}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=4)
grid.fit(X_train_scaled, np.ravel(y_train))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.973 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.973 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=1.000 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.973 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.679 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf']},
             verbose=4)

In [40]:
grid.best_params_

{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}

In [11]:
import csv
import numpy as np
import os
import time

from benchmarks import get_function_details
from src.entities import GSA


# Select optimizers
gsa = True  # Code by Himanshu Mittal

# Select benchmark function
F1 = True

algorithm = [gsa]
objective_function = [F1]

# Select number of repetitions for each experiment. 
# To obtain meaningful statistical results, usually 30 independent runs 
# are executed for each algorithm.
runs = 3

# Select chaotic constant
chaotic_constant = True

# Select general parameters for all optimizers (population size, number of iterations)
population_size = 30
iterations = 100

# Export results ?
export = True

# ExportToFile="YourResultsAreHere.csv"
# Automaticly generated name by date and time

save_path = "data/output/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

ExportToFile = save_path + "experiment" + time.strftime("%Y-%m-%d-%H-%M-%S_") + str(chaotic_constant) + ".csv"

# Check if it works at least once
atLeastOneIteration = False

# CSV Header for the convergence
CnvgHeader = []
# SolHeader=[]

for l in range(iterations):
    CnvgHeader.append("Iter" + str(l + 1))
    # SolHeader.append("Sol_Iter"+str(l+1))

for j in range(len(objective_function)):
    if objective_function[j]:  # start experiment if an Algorithm and an objective function is selected
        for k in range(0, runs):
            
            real_boundaries = [(function_details['lower_bound'], function_details['upper_bound']) for _ in range(function_details['dim'])]
            function_details = get_function_details(j)
            gsa_algo = GSA(objective_function = """function_details['function']""",
                           r_dim=function_details['dim'],
                           d_dim=,  # 
                           boundaries={'real': real_boundaries, 'discrete': []})
            
            gsa_algo.optimize(population_size=population_size,
                              iters=iterations,
                              chaotic_constant=True)

            if export:
                with open(ExportToFile, 'a') as out:
                    writer = csv.writer(out, delimiter=',')
                    if not atLeastOneIteration:  # just one time to write the header of the CSV file
                        header = np.concatenate(
                            [["Optimizer", "objfname", "startTime", "EndTime", "ExecutionTime"], CnvgHeader])
                        print(header)
                        writer.writerow(header)
                    a = np.concatenate(
                        [[gsa_algo.objective_function_name,
                          gsa_algo.start_time,
                          gsa_algo.end_time,
                          gsa_algo.execution_time],
                          gsa_algo.convergence])
                    writer.writerow(a)
                out.close()
            atLeastOneIteration = True  # at least one experiment

if not atLeastOneIteration:  # Faild to run at least one experiment
    print("No Optimizer or Cost function is selected. Check lists of available optimizers and cost functions")

GSA is optimizing  "F1"
['At iteration 1 the best fitness is 8828.851219458797']
['At iteration 2 the best fitness is 5347.581633851479']
['At iteration 3 the best fitness is 1882.4276629741098']
['At iteration 4 the best fitness is 774.1668420656121']
['At iteration 5 the best fitness is 473.62971257280014']
['At iteration 6 the best fitness is 315.3291651826068']
['At iteration 7 the best fitness is 243.04707678436114']
['At iteration 8 the best fitness is 239.6275349930173']
['At iteration 9 the best fitness is 239.6275349930173']
['At iteration 10 the best fitness is 217.17327629610273']
['At iteration 11 the best fitness is 217.17327629610273']
['At iteration 12 the best fitness is 206.62148308794687']
['At iteration 13 the best fitness is 197.8860353059992']
['At iteration 14 the best fitness is 152.85288825048485']
['At iteration 15 the best fitness is 143.18794767067305']
['At iteration 16 the best fitness is 109.6020646845365']
['At iteration 17 the best fitness is 93.97896602

In [ ]:
import pandas as pd

In [3]:
output_file = "../data/output/experiment2024-02-06-11-24-28.csv"

data = pd.read_csv(output_file)

In [4]:
data

,Optimizer,objfname,startTime,EndTime,ExecutionTime,Iter1,Iter2,Iter3,Iter4,Iter5,...,Iter91,Iter92,Iter93,Iter94,Iter95,Iter96,Iter97,Iter98,Iter99,Iter100
0,GSA,F1,2024-02-06-11-24-28,2024-02-06-11-24-29,0.54548,21634.17186,13724.250576,5564.349644,1210.276746,439.623279,...,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882


In [43]:
# Nº de variables discretas:
dim_d = len(X.columns)

# Nº variables reales (C y Gamma):
dim_r = 2

9

In [ ]:
population_size = 5 
iterations = 20

g_zero_real= 100  # OK
alpha = 20  # OK
g_zero_discrete = 1  # OK

fitness_evaluations = 100

In [47]:
# Filter features
# Set C and Gamma
solution = {'real': [0.1, 0.2], 'discrete': np.array([0, 1, 1, 0, 0, 1, 0, 1, 1])}
C, gamma = solution['real']

X_train_scaled_filtered = X_train_scaled.iloc[:, solution['discrete'] == 1]

print(X_train_scaled.columns)
print(X_train_scaled_filtered.columns)

Index(['Clump_thickness', 'Uniformity_of_cell_size',
       'Uniformity_of_cell_shape', 'Marginal_adhesion',
       'Single_epithelial_cell_size', 'Bare_nuclei', 'Bland_chromatin',
       'Normal_nucleoli', 'Mitoses'],
      dtype='object')
Index(['Uniformity_of_cell_size', 'Uniformity_of_cell_shape', 'Bare_nuclei',
       'Normal_nucleoli', 'Mitoses'],
      dtype='object')


In [51]:
svc_model = SVC(C=C, gamma=gamma, kernel="rbf")
svc_model.fit(X_train_scaled_filtered, np.ravel(y_train))
y_predict = svc_model.predict(X_train_scaled_filtered)
cm = confusion_matrix(y_train, y_predict)

cm

array([[360,   7],
       [ 12, 180]])

In [9]:
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from typing import Mapping
from ucimlrepo import fetch_ucirepo

from benchmarks import gsa_svm_fitness

# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
X = X.fillna(value=0)

y = breast_cancer_wisconsin_original.data.targets 

class UCI:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        min_train = X_train.min()
        range_train = (X_train - min_train).max()
        self.X_train_scaled = (X_train - min_train)/range_train
        
        min_test = X_test.min()
        range_test = (X_test - min_test).max()
        self.X_test_scaled = (X_test - min_test)/range_test
    
    def get_fitness(self,
                    solution: Mapping[str, np.ndarray]
                    ) -> float:
        C, gamma = solution['real']
        X_train_scaled_filtered = self.X_train_scaled.iloc[:, solution['discrete'] == 1]
        svc_model = SVC(C=C, gamma=gamma, kernel="rbf")
        svc_model.fit(X_train_scaled_filtered, np.ravel(self.y_train))
        y_predict = svc_model.predict(X_train_scaled_filtered)
        cm = confusion_matrix(self.y_train, y_predict)
        
        return gsa_svm_fitness(conf_matrix=cm, solution=solution)

uci = UCI(X, y)

In [14]:
solution = {'real': [0.1, 1], 'discrete': np.array([0, 1, 1, 0, 0, 1, 0, 1, 1])}
uci.get_fitness(solution=solution)

77.51286026634864

In [19]:
import csv
import numpy as np
import os
import time

from benchmarks import get_function_details
from src.entities import GSA

# Select number of repetitions for each experiment. 
# To obtain meaningful statistical results, usually 30 independent runs 
# are executed for each algorithm.
runs = 3

# Select chaotic constant
chaotic_constant = True

# Export results ?
export = True

save_path = "data/output/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

ExportToFile = save_path + "experiment" + time.strftime("%Y-%m-%d-%H-%M-%S_") + str(chaotic_constant) + ".csv"

# Check if it works at least once
atLeastOneIteration = False

# CSV Header for the convergence
CnvgHeader = []

population_size = 5 
iterations = 20

fitness_evaluations = 100

boundaries = {'real': [(0, 200), (0, 200)], 'discrete': [(0, 1) for _ in range(len(X.columns))]}

for l in range(iterations):
    CnvgHeader.append("Iter" + str(l + 1))
    # SolHeader.append("Sol_Iter"+str(l+1))

for k in range(runs):
    gsa_algo = GSA(objective_function = uci.get_fitness,
                   r_dim=2,
                   d_dim=len(X.columns),
                   boundaries=boundaries)
    
    gsa_algo.optimize(population_size=population_size,
                      iters=iterations,
                      chaotic_constant=True)

    if export:
        with open(ExportToFile, 'a') as out:
            writer = csv.writer(out, delimiter=',')
            if not atLeastOneIteration:  # just one time to write the header of the CSV file
                header = np.concatenate(
                    [["Optimizer", "objfname", "startTime", "EndTime", "ExecutionTime"], CnvgHeader])
                print(header)
                writer.writerow(header)
            a = np.concatenate(
                [[gsa_algo.objective_function_name,
                  gsa_algo.start_time,
                  gsa_algo.end_time,
                  gsa_algo.execution_time],
                  gsa_algo.convergence])
            writer.writerow(a)
        out.close()
    atLeastOneIteration = True  # at least one experiment

if not atLeastOneIteration:  # Faild to run at least one experiment
    print("No Optimizer or Cost function is selected. Check lists of available optimizers and cost functions")

GSA is optimizing  "get_fitness"
['At iteration 1 the best fitness is 78.84531902206322']
['At iteration 2 the best fitness is 78.84531902206322']
['At iteration 3 the best fitness is 78.84531902206322']
['At iteration 4 the best fitness is 78.84531902206322']
['At iteration 5 the best fitness is 78.84531902206322']
['At iteration 6 the best fitness is 78.84531902206322']
['At iteration 7 the best fitness is 78.84531902206322']
['At iteration 8 the best fitness is 78.84531902206322']
['At iteration 9 the best fitness is 78.84531902206322']
['At iteration 10 the best fitness is 78.84531902206322']
['At iteration 11 the best fitness is 78.84531902206322']
['At iteration 12 the best fitness is 78.84531902206322']
['At iteration 13 the best fitness is 78.84531902206322']
['At iteration 14 the best fitness is 78.84531902206322']
['At iteration 15 the best fitness is 78.84531902206322']
['At iteration 16 the best fitness is 78.84531902206322']
['At iteration 17 the best fitness is 78.8453190